In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import getpass

%matplotlib inline

In [2]:
USERNAME = 'mphan'
PASSWORD = getpass.getpass('Enter your password: ')

Enter your password:  ········


In [3]:
DATASET_ID = 'cmems_mod_glo_phy_my_0.083_P1D-m'

In [4]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-
__author__ = "Copernicus Marine User Support Team"
__copyright__ = "(C) 2022 E.U. Copernicus Marine Service Information"
__credits__ = ["E.U. Copernicus Marine Service Information"]
__license__ = "MIT License - You must cite this source"
__version__ = "202104"
__maintainer__ = "D. Bazin, E. DiMedio, C. Giordan"
__email__ = "servicedesk dot cmems at mercator hyphen ocean dot eu"

def copernicusmarine_datastore(dataset, username, password):
    from pydap.client import open_url
    from pydap.cas.get_cookies import setup_session
    cas_url = 'https://cmems-cas.cls.fr/cas/login'
    session = setup_session(cas_url, username, password)
    session.cookies.set("CASTGC", session.cookies.get_dict()['CASTGC'])
    database = ['my', 'nrt']
    url = f'https://{database[0]}.cmems-du.eu/thredds/dodsC/{dataset}'
    try:
        data_store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits 
    except:
        url = f'https://{database[1]}.cmems-du.eu/thredds/dodsC/{dataset}'
        data_store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits
    return data_store

In [5]:
data_store = copernicusmarine_datastore(DATASET_ID, USERNAME, PASSWORD)

In [7]:
DS = xr.open_dataset(data_store)['mlotst'].sel(latitude=slice(-12, 32), longitude=slice(42, 102))
DS

<xarray.DataArray 'mlotst' (time: 10227, latitude: 529, longitude: 721)>
[3900669843 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 42.0 42.08 42.17 42.25 ... 101.8 101.9 102.0
  * latitude   (latitude) float32 -12.0 -11.92 -11.83 ... 31.83 31.92 32.0
  * time       (time) datetime64[ns] 1993-01-01T12:00:00 ... 2020-12-31T12:00:00
Attributes:
    long_name:      Density ocean mixed layer thickness
    standard_name:  ocean_mixed_layer_thickness_defined_by_sigma_theta
    units:          m
    unit_long:      Meters
    valid_min:      1
    valid_max:      6447
    cell_methods:   area: mean
    _ChunkSizes:    [1, 681, 1440]

In [38]:
DS

<xarray.DataArray 'so' (time: 10227, latitude: 529, longitude: 721)>
[3900669843 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 42.0 42.08 42.17 42.25 ... 101.8 101.9 102.0
  * latitude   (latitude) float32 -12.0 -11.92 -11.83 ... 31.83 31.92 32.0
  * time       (time) datetime64[ns] 1993-01-01T12:00:00 ... 2020-12-31T12:00:00
Attributes:
    long_name:      Salinity
    standard_name:  sea_water_salinity
    units:          1e-3
    unit_long:      Practical Salinity Unit
    valid_min:      1
    valid_max:      28336
    cell_methods:   area: mean
    _ChunkSizes:    [1, 7, 341, 720]

In [8]:
# slice
for year in range(2002, 2003):
    for month in range(1, 13):
        DS.sel(time=f'{year}-{month}').to_netcdf(f'tutorials/motu/cmems_mod_glo_phy_my_0.083_P1D-m/{year}{month:02d}.nc')
        print(f'finish exporting {year}/{month:02d}...')

finish exporting 2002/01...
finish exporting 2002/02...
finish exporting 2002/03...
finish exporting 2002/04...
finish exporting 2002/05...
finish exporting 2002/06...
finish exporting 2002/07...
finish exporting 2002/08...
finish exporting 2002/09...
finish exporting 2002/10...
finish exporting 2002/11...
finish exporting 2002/12...


In [14]:
import pandas as pd
import os

months = pd.date_range('1/1/1993', '12/31/2020', freq='M')

PATH = 'tutorials/motu/cmems_mod_glo_phy_my_0.083_P1D-m/'
for m in months:
    file_path = os.path.join(PATH, f"{m.strftime('%Y%m')}.nc")
    if not os.path.exists(file_path):
        DS.sel(time=m.strftime('%Y-%m')).to_netcdf(file_path)
        print('finish exporting ' + m.strftime('%Y-%m'))

finish exporting 1993-06
finish exporting 1994-08
finish exporting 1998-03
finish exporting 1999-01
finish exporting 1999-02
finish exporting 2003-02
finish exporting 2011-01
finish exporting 2011-02
finish exporting 2011-03
finish exporting 2011-04
finish exporting 2011-05
finish exporting 2011-06
finish exporting 2011-07
finish exporting 2011-08
finish exporting 2011-09
finish exporting 2011-10
finish exporting 2011-11
finish exporting 2011-12
finish exporting 2014-03
finish exporting 2017-02
finish exporting 2020-02


In [ ]:
DS.isel(time=0).values